# Calgary Traffic Analysis

In [1]:
import csv
import re
import folium
from folium.features import DivIcon

import pandas as pd
import numpy as np

import city_boundary as cb

In [2]:
# parsing City_Boundary_layer.csv to retrieve a list of boundary coordinates
boundary_coordinates = cb.get_boundary_coordinates('City_Boundary_layer.csv')
coordinate_list = cb.get_coordinate_list(boundary_coordinates)

In [3]:
# finding the north, south, east, and west most coordinates from boundary coordinate list
north = max(coordinate_list[0])
south = min(coordinate_list[0])
east = max(coordinate_list[1])
west = min(coordinate_list[1])

# assigning the four boundary corner coordinates
nw = [north, west]
ne = [north, east]
se = [south, east]
sw = [south, west]

# dataframe containing coordinate locations of the four boundary corners
boundary_corners_df = pd.DataFrame([nw,ne,se,sw],\
                                   index=['NW','NE','SE','SW'],\
                                   columns=['Latitude','Longitude'])

boundary_corners_df

,Latitude,Longitude
NW,51.212425,-114.315796
NE,51.212425,-113.859905
SE,50.842822,-113.859905
SW,50.842822,-114.315796


In [8]:
m = folium.Map(location=[51.0447, -114.0719], zoom_start=10)

# boundary lines
cb.draw_boundary_lines(nw,ne,sw,se,m)

m

In [9]:
# gathering points to plot ployline grid
top_points = cb.get_plot_points(ne,nw)
right_points = cb.get_plot_points(se,ne)
bottom_points = cb.get_plot_points(se,sw)
left_points = cb.get_plot_points(sw,nw)

# drawing grid on folium map
cb.draw_grid_lines(top_points, bottom_points, left_points, right_points, m)

m

In [10]:
# computing cell boundaries and assigning cell locations
cells = cb.get_cell_boundaries(top_points, left_points)
cb.draw_cell_number(cells, m)

m

In [11]:
# dataframe containing cell boundaries
cells_df = pd.DataFrame(cells,\
                        index=np.arange(1, len(cells)+1),\
                        columns=['North Boundary','East Boundary','South Boundary','West Boundary'])

cells_df

,North Boundary,East Boundary,South Boundary,West Boundary
1,51.212425,-114.270207,51.175465,-114.315796
2,51.212425,-114.224618,51.175465,-114.270207
3,51.212425,-114.179029,51.175465,-114.224618
4,51.212425,-114.133440,51.175465,-114.179029
5,51.212425,-114.087851,51.175465,-114.133440
...,...,...,...,...
96,50.879782,-114.042261,50.842822,-114.087851
97,50.879782,-113.996672,50.842822,-114.042261
98,50.879782,-113.951083,50.842822,-113.996672
99,50.879782,-113.905494,50.842822,-113.951083
